<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c1317a0df47c72698e9b209ee16015519fd646e25ca663ef9b6fd7cc2d86451e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-23 11:02:07
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.43 C
-------------------
Today PnL: 93.81 K (0.66%)
Current PnL: -19.14 L (-12.75%)
CY Booked + Current PnL: -7.04 L (-4.69%)
-------------------
Total profit:  2.30 L
Total loss:  -21.44 L
-------------------
Total Booked + Current PnL: 19.94 L (16.2%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.33 L (60.57%)
Deployed:  1.23 C
Current:  1.43 C
CAGR/XIRR %: 8.64%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-8.78,73.0,X-LC,2.86,228571.0,18205.0,10583.0,-0.11,8.65,4.63,13.68,37.0,1.72,1.61,26.61,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,2.98,61.0,X-MC,2.42,216030.0,24288.0,13480.0,-1.07,12.67,6.24,19.69,99.0,1.80,1.52,20.33,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-8.63,47.0,X-MC,3.48,90330.0,-348.0,15257.0,-0.48,-0.38,16.89,16.44,101.0,-0.02,0.64,19.71,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,96.80,51.0,M-SC,1.90,84790.0,-15987.0,16076.0,0.40,-15.86,18.96,0.09,245.0,-0.99,0.60,11.40,OX40N,NTT,DURABLES
42,ITC,452.0,-36.66,66.0,X-LC,2.11,204793.0,4655.0,16240.0,1.44,2.33,7.93,10.44,4.0,0.29,1.44,8.02,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BSOFT,836.99,-12.47,65.0,H-SC,15.10,98200.0,-42453.0,115228.0,9.70,-30.18,117.34,51.74,131.0,-0.37,0.69,13.62,XR,ATH,IT
82,VAIBHAVGBL,521.00,56.47,62.0,H-SC,4.54,135529.0,-47246.0,158311.0,6.82,-25.85,116.81,60.77,125.0,-0.30,0.95,21.39,XR,NTT,JEWELLERY
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.69,209260.0,29452.0,82951.0,4.45,16.38,39.64,62.52,88.0,0.36,1.47,56.05,XR,NTT,BANKS
27,HAPPSTMNDS,1488.71,-22.73,49.0,H-SC,16.62,83480.0,-44685.0,154713.0,4.25,-34.87,185.33,85.85,132.0,-0.29,0.59,6.57,AR,ATH,IT
30,HCLTECH,1922.01,5.81,69.0,X-LC,7.58,241544.0,-380.0,60217.0,3.53,-0.16,24.93,24.73,8.0,-0.01,1.70,15.05,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-41.44,47.0,H-SC,18.18,88730.0,-13770.0,54374.0,-2.32,-13.43,61.28,39.61,114.0,-0.25,0.62,8.37,AR,ATH,MISC
71,SYMPHONY,1306.00,-44.18,47.0,M-SC,18.66,118771.0,-52370.0,52319.0,-2.16,-30.60,44.05,-0.03,196.0,-1.00,0.84,3.55,OX40N,NTT,DURABLES
72,TANLA,1963.11,-28.54,34.0,H-SC,17.10,191388.0,-79373.0,409322.0,-1.89,-29.31,213.87,121.86,133.0,-0.19,1.35,49.72,AR,ATH,IT
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125383.0,86.0,58842.0,-1.49,0.07,46.93,47.03,141.0,0.00,0.88,56.99,MH,ATH,POWER
37,INDIAMART,4810.62,-55.57,41.0,H-SC,7.35,124179.0,843.0,130785.0,-1.42,0.68,105.32,106.72,119.0,0.01,0.87,24.22,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,41.94,48.0,M-MC,10.31,224204.0,-758.0,168422.0,0.19,-0.34,75.12,74.53,192.0,-0.00,1.58,31.81,XY24,BTT,STEEL
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125383.0,86.0,58842.0,-1.49,0.07,46.93,47.03,141.0,0.00,0.88,56.99,MH,ATH,POWER
37,INDIAMART,4810.62,-55.57,41.0,H-SC,7.35,124179.0,843.0,130785.0,-1.42,0.68,105.32,106.72,119.0,0.01,0.87,24.22,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,120.54,67.0,H-SC,14.39,139374.0,-3105.0,22620.0,2.75,-2.18,16.23,13.70,163.0,-0.14,0.98,64.50,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.69,62.0,H-SC,2.20,229662.0,-40005.0,76340.0,-0.65,-14.83,33.24,13.47,138.0,-0.52,1.62,16.78,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,96.80,51.0,M-SC,1.90,84790.0,-15987.0,16076.0,0.40,-15.86,18.96,0.09,245.0,-0.99,0.60,11.40,OX40N,NTT,DURABLES
67,SIS,528.0,2078.24,53.0,H-SC,3.43,87898.0,-23134.0,46744.0,0.04,-20.84,53.18,21.26,156.0,-0.49,0.62,18.76,OX40N,NTT,MISC
73,TATAELXSI,9161.0,-23.37,50.0,H-MC,7.63,103598.0,-28237.0,70457.0,1.83,-21.42,68.01,32.03,98.0,-0.40,0.73,16.04,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,47.87,56.0,M-LC,5.83,209000.0,5830.0,36303.0,0.76,2.87,17.37,20.74,55.0,0.16,1.47,8.57,XY25,NTT,FINANCE
32,HINDZINC,730.22,28.52,51.0,M-LC,10.62,209996.0,4920.0,107644.0,-0.23,2.40,51.26,54.89,52.0,0.05,1.48,27.07,X5K,ATH,METALS
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125383.0,86.0,58842.0,-1.49,0.07,46.93,47.03,141.0,0.00,0.88,56.99,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,63.0,M-LC,9.04,163728.0,22888.0,21596.0,1.65,16.25,13.19,31.59,66.0,1.06,1.15,47.04,XY24,NTT,BANKS
87,WIPRO,420.00,-12.89,51.0,M-LC,6.22,153016.0,2071.0,107387.0,2.25,1.37,70.18,72.51,53.0,0.02,1.08,7.48,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.69,209260.0,29452.0,82951.0,4.45,16.38,39.64,62.52,88.0,0.36,1.47,56.05,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.39,66.0,M-SC,3.99,104220.0,12722.0,66034.0,-0.57,13.90,63.36,86.07,223.0,0.19,0.73,51.35,XR,NTT,DURABLES
32,HINDZINC,730.22,28.52,51.0,M-LC,10.62,209996.0,4920.0,107644.0,-0.23,2.40,51.26,54.89,52.0,0.05,1.48,27.07,X5K,ATH,METALS
87,WIPRO,420.00,-12.89,51.0,M-LC,6.22,153016.0,2071.0,107387.0,2.25,1.37,70.18,72.51,53.0,0.02,1.08,7.48,XR,NTT,IT
37,INDIAMART,4810.62,-55.57,41.0,H-SC,7.35,124179.0,843.0,130785.0,-1.42,0.68,105.32,106.72,119.0,0.01,0.87,24.22,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,17.0,X-LC,13.40,147723.0,-128800.0,238868.0,1.27,-46.58,161.70,39.81,54.0,-0.54,1.04,4.12,XY24,NTT,AUTO
53,QUESS,986.00,-47.66,36.0,X-SC,38.76,51794.0,-13212.0,164140.0,-0.94,-20.32,316.91,232.18,198.0,-0.08,0.36,1.03,XY24,NTT,MISC
24,DMART,5391.45,-15.93,38.0,X-LC,10.18,102816.0,230.0,26578.0,0.22,0.22,25.85,26.13,38.0,0.01,0.72,25.85,X40N,ATH,FMCG
29,HAVELLS,2069.16,-6.46,43.0,X-MC,2.84,231691.0,-16040.0,91101.0,0.07,-6.47,39.32,30.30,92.0,-0.18,1.63,5.96,X40,ATH,ELECTRICAL
3,ACC,3906.00,-38.07,46.0,X-MC,3.04,183790.0,-54161.0,206819.0,-0.59,-22.76,112.53,64.15,174.0,-0.26,1.29,3.10,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.76,50.0,H-LC,0.68,157780.0,-28315.0,75798.0,1.70,-15.22,48.04,25.51,15.0,-0.37,1.11,16.27,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,66.0,X-LC,2.11,204793.0,4655.0,16240.0,1.44,2.33,7.93,10.44,4.0,0.29,1.44,8.02,X40,NTT,FMCG
85,VOLTAS,1530.00,2.98,61.0,X-MC,2.42,216030.0,24288.0,13480.0,-1.07,12.67,6.24,19.69,99.0,1.80,1.52,20.33,XY25,NTT,AC
29,HAVELLS,2069.16,-6.46,43.0,X-MC,2.84,231691.0,-16040.0,91101.0,0.07,-6.47,39.32,30.30,92.0,-0.18,1.63,5.96,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-8.78,73.0,X-LC,2.86,228571.0,18205.0,10583.0,-0.11,8.65,4.63,13.68,37.0,1.72,1.61,26.61,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,36.0,X-SC,38.76,51794.0,-13212.0,164140.0,-0.94,-20.32,316.91,232.18,198.0,-0.08,0.36,1.03,XY24,NTT,MISC
3,ACC,3906.00,-38.07,46.0,X-MC,3.04,183790.0,-54161.0,206819.0,-0.59,-22.76,112.53,64.15,174.0,-0.26,1.29,3.10,XY24,BTT,CEMENT
74,TATAMOTORS,1065.00,-53.17,17.0,X-LC,13.40,147723.0,-128800.0,238868.0,1.27,-46.58,161.70,39.81,54.0,-0.54,1.04,4.12,XY24,NTT,AUTO
10,BAJAJHFL,181.50,-14.06,47.0,X-MC,5.67,180925.0,-19581.0,116733.0,-0.34,-9.77,64.52,48.45,90.0,-0.17,1.27,4.33,X40N,ATH,FINANCE
29,HAVELLS,2069.16,-6.46,43.0,X-MC,2.84,231691.0,-16040.0,91101.0,0.07,-6.47,39.32,30.30,92.0,-0.18,1.63,5.96,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,36.0,X-SC,38.76,51794.0,-13212.0,164140.0,-0.94,-20.32,316.91,232.18,198.0,-0.08,0.36,1.03,XY24,NTT,MISC
51,PAGEIND,51769.93,-25.96,47.0,X-MC,6.66,83670.0,160.0,19872.0,0.60,0.19,23.75,23.98,82.0,0.01,0.59,6.02,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.01,47.0,X-SC,4.75,83190.0,-61970.0,137896.0,-0.02,-42.69,165.76,52.31,136.0,-0.45,0.59,11.35,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-8.63,47.0,X-MC,3.48,90330.0,-348.0,15257.0,-0.48,-0.38,16.89,16.44,101.0,-0.02,0.64,19.71,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-34.87,49.0,X-SC,4.29,97475.0,-31195.0,76489.0,0.76,-24.24,78.47,35.20,219.0,-0.41,0.69,11.88,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-27.08,59.0,X-LC,12.50,290310.0,-55646.0,122337.0,2.72,-16.08,42.14,19.28,1.0,-0.45,2.04,7.32,X40,ATH,IT
40,INFY,2275.00,-19.60,48.0,X-LC,7.59,313621.0,733.0,170955.0,0.79,0.23,54.51,54.87,3.0,0.00,2.21,6.90,X40,BTT,IT
42,ITC,452.00,-36.66,66.0,X-LC,2.11,204793.0,4655.0,16240.0,1.44,2.33,7.93,10.44,4.0,0.29,1.44,8.02,X40,NTT,FMCG
84,VBL,671.64,-16.73,59.0,X-LC,10.10,299732.0,-16110.0,128765.0,2.25,-5.10,42.96,35.67,5.0,-0.13,2.11,8.05,X40N,ATH,FMCG
1,ABB,7934.00,-39.57,56.0,H-LC,6.49,252504.0,-9115.0,128323.0,0.39,-3.48,50.82,45.57,7.0,-0.07,1.78,7.44,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7272.22,72.0,L-SC,7.01,80860.0,-12950.0,89569.0,0.82,-13.80,110.77,81.67,269.0,-0.14,0.57,56.59,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,51.0,L-SC,26.66,81984.0,-31565.0,71613.0,-0.81,-27.80,87.35,35.27,268.0,-0.44,0.58,100.66,XR,NTT,HOTELS
49,MASFIN,398.61,-17.34,57.0,H-SC,9.56,93750.0,-4230.0,25838.0,1.43,-4.32,27.56,22.05,152.0,-0.16,0.66,36.27,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-38.39,66.0,M-SC,3.99,104220.0,12722.0,66034.0,-0.57,13.90,63.36,86.07,223.0,0.19,0.73,51.35,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.18,63.0,H-SC,5.28,125383.0,86.0,58842.0,-1.49,0.07,46.93,47.03,141.0,0.00,0.88,56.99,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-190.91,75.0,M-SC,22.62,157653.0,8433.0,135739.0,0.03,5.65,86.10,96.62,208.0,0.06,1.11,70.61,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,120.54,67.0,H-SC,14.39,139374.0,-3105.0,22620.0,2.75,-2.18,16.23,13.70,163.0,-0.14,0.98,64.50,OX40N,NTT,BANKS
5,ANGELONE,3033.00,16.84,71.0,X-SC,7.02,206624.0,15618.0,42089.0,0.88,8.18,20.37,30.21,157.0,0.37,1.46,30.76,X40N,NTT,FINANCE
68,SONACOMS,806.63,-33.12,69.0,M-SC,10.26,83300.0,-17859.0,57860.0,3.04,-17.65,69.46,39.54,202.0,-0.31,0.59,17.46,AR,ATH,AUTO
59,REPCOHOME,880.00,-55.29,72.0,H-SC,2.53,257088.0,-29417.0,286756.0,2.11,-10.27,111.54,89.82,134.0,-0.10,1.81,36.02,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.52
1,25,43.90
2,50,74.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.33
LC    32.85
MC    22.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.29
X40      15.98
XY25     12.32
XR       11.28
X40N      9.34
AR        7.93
OX40N     7.60
X200      1.79
X5K       1.48
SR        1.09
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.54
X-LC    21.97
X-MC    17.05
M-SC    12.66
M-LC     5.18
H-LC     4.68
X-SC     4.63
H-MC     3.86
M-MC     1.58
L-SC     1.50
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.81,-2.46,35.90
IT,12.70,-16.61,79.55
FINANCE,12.18,-9.34,58.90
MISC,6.85,-17.93,83.27
BANKS,6.59,-6.62,62.71
PAINTS,5.63,-15.39,32.59
ELECTRICAL,5.29,-9.69,48.17
INSURANCE,3.92,0.60,36.46
AUTO,2.81,-44.00,104.10


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3271907.0,22
XR,1307930.0,14
AR,1168971.0,9
X40,776142.0,12
X40N,602882.0,8
OX40N,569164.0,10
XY25,432260.0,8
SR,266077.0,2
X5K,107644.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3374350.0,25
M-SC,1207242.0,15
X-MC,1113652.0,13
X-LC,1016896.0,13
X-SC,524582.0,6
H-MC,331964.0,3
H-LC,275610.0,3
M-LC,272930.0,4
L-SC,249411.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1160238.0      6
           AR         840656.0      5
           XR         784353.0      7
M-SC       XY24       688103.0      6
X-MC       XY24       587393.0      4
X-LC       X40        451743.0      5
           XY24       303459.0      2
X-SC       X40N       283953.0      4
H-SC       SR         266077.0      2
           OX40N      264184.0      4
X-MC       X40        247910.0      6
H-LC       AR         204121.0      2
X-LC       X40N       202196.0      3
H-MC       XY24       178556.0      1
M-MC       XY24       168422.0      1
X-SC       XY24       164140.0      1
X-MC       XY25       161616.0      2
L-SC       XR         161182.0      2
M-SC       OX40N      146294.0      4
           XY25       135739.0      1
           AR         124194.0      2
X-MC       X40N       116733.0      1
M-SC       XR         112912.0      2
M-LC       X5K        107644.0      1
           XR         107387.0      1
L-SC       OX40N       88229.0      1
H-MC       XR          82951.0      1
X-SC       X40         76489.0      1
H-LC       X200        71489.0      1
H-MC       OX40N       70457.0      1
X-LC       XY25        59498.0      3
L-MC       XR          59145.0      1
H-SC       MH          58842.0      1
L-LC       XY25        39104.0      1
M-LC       XY25        36303.0      1
           XY24        21596.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
